# Extracción de Entidades y Relaciones desde PDF

Este notebook procesa un archivo PDF para extraer entidades y relaciones usando técnicas de NLP (spaCy).

## Objetivo
- Extraer texto del PDF
- Identificar entidades (Personas, Lugares, Organizaciones, etc.)
- Extraer relaciones entre entidades
- Preparar datos para cargar en un grafo de conocimiento (Neo4j)


## 1. Instalación de dependencias

Primero necesitamos instalar las librerías necesarias:
- `spacy`: Para NLP y extracción de entidades
- `pdfplumber` o `PyPDF2`: Para extraer texto del PDF
- `pandas`: Para manejar datos
- `nltk`: Para tokenización y stemming


In [52]:
# Instalar dependencias (ejecutar solo la primera vez)
# !pip install spacy pdfplumber pandas nltk
# !python -m spacy download en_core_web_lg


## 2. Importar librerías


In [26]:
import os
import re
import json
from collections import Counter

import pdfplumber
import pandas as pd
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from slugify import slugify

# Descargar recursos de NLTK si no están disponibles
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Descargando recursos de NLTK (punkt_tab)...")
    nltk.download('punkt_tab', quiet=True)

print("Librerías importadas correctamente")


Librerías importadas correctamente


## 3. Cargar modelo de spaCy


In [25]:
# Cargar modelo de spaCy (modelo grande para mejor precisión)
# 
# Modelos disponibles:
# - Inglés: 'en_core_web_sm', 'en_core_web_md', 'en_core_web_lg', 'en_core_web_trf'
# - Español: 'es_core_news_sm', 'es_core_news_md', 'es_core_news_lg'

nlp = spacy.load('es_core_news_lg')
print("✓ Modelo spaCy cargado: es_core_news_lg (español)")


stemmer = PorterStemmer()


✓ Modelo spaCy cargado: es_core_news_lg (español)


## 4. Extraer texto del PDF


In [27]:
# Ruta al PDF
pdf_path = "/Users/mflores/Downloads/archivo.pdf"
pdf_path = "/Users/mflores/Downloads/aws.pdf"

import warnings
import logging

# Suprimir warnings de pdfplumber sobre colores inválidos (no afectan la extracción de texto)
warnings.filterwarnings('ignore', message='.*Cannot set gray stroke color.*')
warnings.filterwarnings('ignore', category=UserWarning, module='pdfplumber')

# También suprimir logs de pdfplumber relacionados con colores
logging.getLogger('pdfplumber').setLevel(logging.ERROR)

def extract_text_from_pdf(pdf_path, show_progress=True):
    """
    Extrae todo el texto de un PDF, asegurando que se lean todas las páginas.
    
    Args:
        pdf_path (str): Ruta al archivo PDF
        show_progress (bool): Mostrar progreso de extracción
    
    Returns:
        tuple: (texto_completo, estadísticas_dict)
            - texto_completo: Todo el texto extraído
            - estadísticas: Diccionario con info sobre páginas procesadas
    """
    text = ""
    stats = {
        'total_pages': 0,
        'pages_with_text': 0,
        'pages_empty': 0,
        'pages_with_errors': 0,
        'total_chars': 0
    }
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Obtener número total de páginas
            stats['total_pages'] = len(pdf.pages)
            
            if show_progress:
                print(f"📄 PDF abierto: {stats['total_pages']} páginas detectadas")
                print("=" * 60)
            
            # Procesar cada página
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    page_text = page.extract_text()
                    
                    if page_text and page_text.strip():
                        text += page_text + "\n"
                        stats['pages_with_text'] += 1
                        stats['total_chars'] += len(page_text)
                    else:
                        stats['pages_empty'] += 1
                        if show_progress and page_num <= 10:  # Mostrar solo primeras 10 vacías
                            print(f"⚠️  Página {page_num}: Sin texto extraído")
                    
                    # Mostrar progreso cada 50 páginas o al final
                    if show_progress and (page_num % 50 == 0 or page_num == stats['total_pages']):
                        print(f"  Procesando... Página {page_num}/{stats['total_pages']} "
                              f"({page_num/stats['total_pages']*100:.1f}%)")
                
                except Exception as e:
                    stats['pages_with_errors'] += 1
                    if show_progress:
                        print(f"❌ Error en página {page_num}: {str(e)}")
                    # Continuar con la siguiente página aunque haya error
                    continue
            
            stats['total_chars'] = len(text)
            
    except Exception as e:
        print(f"❌ Error al abrir el PDF: {str(e)}")
        raise
    
    return text, stats

# Extraer texto del PDF
print("=" * 60)
print("EXTRACCIÓN DE TEXTO DEL PDF")
print("=" * 60)
print(f"📄 Archivo: {pdf_path}")
print()

pdf_text, extraction_stats = extract_text_from_pdf(pdf_path, show_progress=True)

# Mostrar estadísticas
print("\n" + "=" * 60)
print("📊 ESTADÍSTICAS DE EXTRACCIÓN")
print("=" * 60)
print(f"Total de páginas en el PDF: {extraction_stats['total_pages']}")
print(f"Páginas con texto extraído: {extraction_stats['pages_with_text']}")
print(f"Páginas vacías (sin texto): {extraction_stats['pages_empty']}")
print(f"Páginas con errores: {extraction_stats['pages_with_errors']}")
print(f"Total de caracteres extraídos: {extraction_stats['total_chars']:,}")
print(f"Promedio de caracteres por página: {extraction_stats['total_chars']//max(extraction_stats['pages_with_text'], 1):,}")

# Verificación de completitud
print("\n" + "=" * 60)
print("✅ VERIFICACIÓN")
print("=" * 60)
if extraction_stats['pages_with_text'] == extraction_stats['total_pages']:
    print("✓ Todas las páginas fueron procesadas exitosamente")
elif extraction_stats['pages_with_text'] + extraction_stats['pages_empty'] == extraction_stats['total_pages']:
    print("✓ Todas las páginas fueron procesadas (algunas pueden estar vacías)")
else:
    print(f"⚠️  Advertencia: Solo se procesaron {extraction_stats['pages_with_text']} de {extraction_stats['total_pages']} páginas")

# Mostrar muestra del texto
print("\n" + "=" * 60)
print("📝 MUESTRA DEL TEXTO EXTRAÍDO")
print("=" * 60)
print(f"Primeros 500 caracteres:\n{pdf_text[:500]}...")
print(f"\nÚltimos 500 caracteres:\n...{pdf_text[-500:]}")


EXTRACCIÓN DE TEXTO DEL PDF
📄 Archivo: /Users/mflores/Downloads/aws.pdf

📄 PDF abierto: 170 páginas detectadas
  Procesando... Página 50/170 (29.4%)
  Procesando... Página 100/170 (58.8%)
  Procesando... Página 150/170 (88.2%)
  Procesando... Página 170/170 (100.0%)

📊 ESTADÍSTICAS DE EXTRACCIÓN
Total de páginas en el PDF: 170
Páginas con texto extraído: 170
Páginas vacías (sin texto): 0
Páginas con errores: 0
Total de caracteres extraídos: 409,983
Promedio de caracteres por página: 2,411

✅ VERIFICACIÓN
✓ Todas las páginas fueron procesadas exitosamente

📝 MUESTRA DEL TEXTO EXTRAÍDO
Primeros 500 caracteres:
AWS Documento técnico
Descripción general de Amazon Web
Services
Copyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.
Descripción general de Amazon Web Services AWS Documento técnico
Descripción general de Amazon Web Services: AWS Documento técnico
Copyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.
Las marcas com

## 5. Limpiar y preprocesar texto


In [28]:
def clean_text(text):
    """Limpia el texto extraído del PDF"""
    # Normalizar espacios en blanco
    text = re.sub(r'\s+', ' ', text)
    # Eliminar saltos de línea múltiples
    text = re.sub(r'\n\s*\n', '\n', text)
    return text.strip()

cleaned_text = clean_text(pdf_text)
print(f"Texto limpio: {len(cleaned_text)} caracteres")


Texto limpio: 409982 caracteres


## 6. Extraer entidades (NER - Named Entity Recognition)


In [29]:
def extract_entities(text, nlp_model):
    """Extrae todas las entidades del texto usando spaCy NER"""
    doc = nlp_model(text)
    entities = []
    
    for ent in doc.ents:
        entities.append({
            'text': ent.text,
            'label': ent.label_,
            'start': ent.start_char,
            'end': ent.end_char,
            'description': spacy.explain(ent.label_)
        })
    
    return entities, doc

entities, doc = extract_entities(cleaned_text, nlp) 

# Mostrar entidades encontradas
entities_df = pd.DataFrame(entities)
print(f"Total de entidades encontradas: {len(entities)}")
print("\nTipos de entidades:")
print(entities_df['label'].value_counts())
print("\nPrimeras 10 entidades:")
print(entities_df.head(10))


Total de entidades encontradas: 4098

Tipos de entidades:
label
MISC    3339
ORG      461
LOC      189
PER      109
Name: count, dtype: int64

Primeras 10 entidades:
                                                text label  start  end  \
0                                      AWS Documento  MISC      0   13   
1  Descripción general de Amazon Web Services Cop...  MISC     22   74   
2                                Amazon Web Services  MISC     82  101   
3                                All rights reserved   ORG    131  150   
4  Descripción general de Amazon Web Services AWS...  MISC    152  208   
5         Descripción general de Amazon Web Services  MISC    217  259   
6                                          Copyright  MISC    283  292   
7                                Amazon Web Services  MISC    300  319   
8                                All rights reserved   ORG    349  368   
9       Las marcas comerciales y la imagen comercial  MISC    370  414   

                   

## 7. Definir funciones de extracción de relaciones

Basado en el código del proyecto original, extraemos relaciones usando análisis de dependencias sintácticas.


In [30]:
def filter_entity(token):
    """Filtra tokens que no son entidades válidas"""
    if token.ent_type_ is None or token.ent_type_ == '':
        return False
    if token.is_punct:
        return False
    if token.text.startswith("http"):
        return False
    return True

def extract_relations_from_sentence_improved(sentence, nlp_model):
    """
    Extrae relaciones de una oración usando múltiples métodos mejorados.
    
    Esta versión mejorada:
    1. Usa las entidades ya identificadas por spaCy (doc.ents)
    2. Busca relaciones entre pares de entidades
    3. Identifica palabras de relación (verbos, preposiciones) entre entidades
    4. Usa análisis sintáctico para mejorar la precisión
    
    Args:
        sentence (str): Oración completa a analizar
        nlp_model: Modelo de spaCy cargado
    
    Returns:
        list: Lista de diccionarios con relaciones extraídas
    """
    doc = nlp_model(sentence)
    relations = []
    
    # Obtener todas las entidades en la oración
    entities = list(doc.ents)
    
    # MÉTODO PRINCIPAL: Extraer relaciones entre pares de entidades
    if len(entities) >= 2:
        for i in range(len(entities)):
            for j in range(i + 1, len(entities)):
                ent1 = entities[i]
                ent2 = entities[j]
                
                # Encontrar la palabra de relación entre las entidades
                # Buscar tokens entre las dos entidades
                start_pos = ent1.end_char
                end_pos = ent2.start_char
                
                # Encontrar verbos, preposiciones o palabras clave entre las entidades
                relation_words = []
                for token in doc:
                    token_start = token.idx
                    token_end = token.idx + len(token.text)
                    
                    # Si el token está entre las dos entidades
                    if start_pos <= token_start < end_pos or start_pos < token_end <= end_pos:
                        # Priorizar verbos y preposiciones
                        if token.pos_ in ('VERB', 'AUX', 'ADP', 'SCONJ'):
                            relation_words.append(token.text)
                        # También considerar palabras comunes de relación
                        elif token.text.lower() in ('es', 'son', 'fue', 'fueron', 'está', 'están', 
                                                    'de', 'en', 'con', 'para', 'por', 'desde', 'hasta',
                                                    'opera', 'opera', 'tiene', 'tienen', 'produce', 'producen'):
                            relation_words.append(token.text)
                
                # Si no hay palabras explícitas, buscar el verbo principal de la oración
                if not relation_words:
                    # Buscar el verbo raíz de la oración
                    root = [token for token in doc if token.dep_ == 'ROOT']
                    if root and root[0].pos_ in ('VERB', 'AUX'):
                        relation_words.append(root[0].text)
                
                # Si aún no hay relación, usar una palabra genérica basada en proximidad
                if not relation_words:
                    # Buscar cualquier palabra entre las entidades
                    for token in doc:
                        if start_pos <= token.idx < end_pos:
                            relation_words.append(token.text)
                            break
                
                # Usar la primera palabra de relación encontrada, o una genérica
                relation = ' '.join(relation_words[:2]) if relation_words else 'relacionado_con'
                
                # Determinar dirección de la relación (sujeto → objeto)
                # La entidad que aparece primero es generalmente el sujeto
                if ent1.start_char < ent2.start_char:
                    subject = ent1
                    object_ent = ent2
                else:
                    subject = ent2
                    object_ent = ent1
                
                relations.append({
                    'subject': subject.text,
                    'subject_type': subject.label_,
                    'relation': relation,
                    'object': object_ent.text,
                    'object_type': object_ent.label_,
                    'dependency': 'entity_pair'
                })
    
    # MÉTODO COMPLEMENTARIO: Usar análisis de dependencias sintácticas
    # Este método es más preciso pero más restrictivo
    for token in doc:
        # Relaciones preposicionales (ej: "X en Y", "X de Y")
        if token.dep_ == 'pobj' and token.ent_type_:
            prep_head = token.head  # La preposición
            if prep_head and prep_head.head:
                head_entity = prep_head.head
                # Verificar si el head es parte de una entidad
                for ent in entities:
                    if ent.start_char <= head_entity.idx < ent.end_char:
                        relations.append({
                            'subject': ent.text,
                            'subject_type': ent.label_,
                            'relation': prep_head.text,
                            'object': token.text,
                            'object_type': token.ent_type_,
                            'dependency': token.dep_
                        })
                        break
        
        # Relaciones de objeto directo (ej: "X hace Y")
        if token.dep_ == 'dobj' and token.ent_type_:
            # Buscar el sujeto del verbo
            verb = token.head
            if verb:
                # Buscar sujetos del verbo
                subjects = [w for w in verb.lefts if w.dep_ in ('nsubj', 'nsubjpass')]
                for subj in subjects:
                    # Verificar si el sujeto es parte de una entidad
                    for ent in entities:
                        if ent.start_char <= subj.idx < ent.end_char:
                            relations.append({
                                'subject': ent.text,
                                'subject_type': ent.label_,
                                'relation': verb.text,
                                'object': token.text,
                                'object_type': token.ent_type_,
                                'dependency': token.dep_
                            })
                            break
    
    return relations

# Función principal que usa el método mejorado
def extract_relations_from_sentence(sentence, nlp_model):
    """
    Extrae relaciones de una oración usando el método mejorado.
    
    Esta función usa múltiples métodos para encontrar relaciones:
    1. Análisis de pares de entidades (método principal)
    2. Análisis de dependencias sintácticas (método complementario)
    
    Args:
        sentence (str): Oración completa a analizar
        nlp_model: Modelo de spaCy cargado
    
    Returns:
        list: Lista de diccionarios con relaciones extraídas
    """
    return extract_relations_from_sentence_improved(sentence, nlp_model)


## 8. Procesar todo el documento

### 🔍 ¿Cómo se definen las oraciones (`sentences`)?

**Según la literatura de NLP y mejores prácticas:**

1. **Segmentación de oraciones**: El texto debe dividirse en oraciones completas antes de extraer relaciones
   - Cada oración debe tener estructura sintáctica completa (sujeto, verbo, objeto)
   - Las oraciones incompletas producen relaciones incorrectas o faltantes

2. **Métodos de segmentación**:
   - **NLTK `sent_tokenize`**: Basado en reglas y puntuación (más simple, pero puede fallar con casos complejos)
   - **spaCy segmentador nativo**: Usa el mismo modelo de lenguaje, más preciso y consistente
   - **Recomendación**: Usar el segmentador de spaCy porque está optimizado para el mismo modelo que analiza dependencias

3. **Ventajas de usar spaCy para segmentación**:
   - ✅ Consistencia: Mismo modelo para segmentación y análisis sintáctico
   - ✅ Precisión: Mejor manejo de abreviaciones, números con puntos, etc.
   - ✅ Optimización: Oraciones segmentadas están listas para dependency parsing
   - ✅ Multilenguaje: Funciona mejor con español que NLTK

4. **Proceso recomendado**:
   ```
   Texto completo → Procesar con spaCy → Obtener oraciones (doc.sents) → 
   Analizar cada oración → Extraer relaciones
   ```

**Referencias:**
- Manning et al. (2014): "Foundations of Statistical Natural Language Processing"
- spaCy Documentation: "Sentence Segmentation"
- Jurafsky & Martin (2020): "Speech and Language Processing"


In [31]:
def process_full_text(text, nlp_model, max_chars=None, use_spacy_segmentation=True):
    """
    Procesa el texto completo y extrae todas las relaciones.
    
    IMPORTANTE: Cómo se definen las oraciones:
    - Si use_spacy_segmentation=True (recomendado): Usa el segmentador nativo de spaCy
      * Procesa el texto completo con spaCy una vez
      * spaCy divide automáticamente en oraciones usando el modelo de lenguaje
      * Más preciso porque usa el mismo modelo para segmentación y análisis sintáctico
      * Mejor manejo de casos complejos (abreviaciones, números, etc.)
    
    - Si use_spacy_segmentation=False: Usa NLTK sent_tokenize
      * Basado en reglas de puntuación
      * Puede fallar con casos complejos
      * No está optimizado para el modelo de spaCy
    
    Args:
        text (str): Texto completo a procesar
        nlp_model: Modelo de spaCy cargado
        max_chars (int, optional): Límite de caracteres a procesar (None = todo el texto)
        use_spacy_segmentation (bool): Usar segmentador de spaCy (True) o NLTK (False)
    
    Returns:
        tuple: (relaciones, entidades)
    """
    # Limitar tamaño si se especifica
    if max_chars and len(text) > max_chars:
        text_to_process = text[:max_chars]
        print(f"⚠️  Procesando solo primeros {max_chars:,} caracteres de {len(text):,} totales")
    else:
        text_to_process = text
        print(f"✓ Procesando todo el texto: {len(text_to_process):,} caracteres")
    
    all_relations = []
    all_entities_list = []
    sentences_list = []  # Para almacenar oraciones según el método usado
    
    if use_spacy_segmentation:
        # MÉTODO RECOMENDADO: Usar segmentador nativo de spaCy
        print("\n📝 Usando segmentador de spaCy (método recomendado)")
        print("=" * 60)
        
        # Procesar todo el texto con spaCy (eficiente: lo hace una sola vez)
        # spaCy automáticamente divide en oraciones y las almacena en doc.sents
        print("Procesando texto completo con spaCy...")
        doc = nlp_model(text_to_process)
        
        # Obtener oraciones del documento procesado
        sentences_list = list(doc.sents)
        print(f"✓ Texto dividido en {len(sentences_list)} oraciones por spaCy")
        
        print(f"\nProcesando {len(sentences_list)} oraciones...")
        
        for i, sent in enumerate(sentences_list):
            if i % 100 == 0:
                print(f"  Procesando oración {i}/{len(sentences_list)} "
                      f"({i/len(sentences_list)*100:.1f}%)")
            
            # La oración ya está procesada como parte del doc, pero podemos procesarla
            # individualmente para extraer entidades y relaciones
            sentence_text = sent.text.strip()
            
            # Extraer entidades de la oración
            for ent in sent.ents:
                all_entities_list.append({
                    'text': ent.text,
                    'label': ent.label_,
                    'sentence_idx': i
                })
            
            # Extraer relaciones (procesa la oración individualmente)
            relations = extract_relations_from_sentence(sentence_text, nlp_model)
            for rel in relations:
                rel['sentence_idx'] = i
                rel['sentence'] = sentence_text
                all_relations.append(rel)
    
    else:
        # MÉTODO ALTERNATIVO: Usar NLTK sent_tokenize
        print("\n📝 Usando NLTK sent_tokenize (método alternativo)")
        print("=" * 60)
        print("⚠️  Nota: Este método puede ser menos preciso que spaCy")
        
        # Dividir en oraciones usando NLTK
        sentences_list = sent_tokenize(text_to_process)
        print(f"✓ Texto dividido en {len(sentences_list)} oraciones por NLTK")
        
        print(f"\nProcesando {len(sentences_list)} oraciones...")
        
        for i, sentence in enumerate(sentences_list):
            if i % 100 == 0:
                print(f"  Procesando oración {i}/{len(sentences_list)} "
                      f"({i/len(sentences_list)*100:.1f}%)")
            
            # Procesar cada oración con spaCy
            doc = nlp_model(sentence)
            
            # Extraer entidades
            for ent in doc.ents:
                all_entities_list.append({
                    'text': ent.text,
                    'label': ent.label_,
                    'sentence_idx': i
                })
            
            # Extraer relaciones
            relations = extract_relations_from_sentence(sentence, nlp_model)
            for rel in relations:
                rel['sentence_idx'] = i
                rel['sentence'] = sentence
                all_relations.append(rel)
    
    # Estadísticas finales
    total_sentences = len(sentences_list) if use_spacy_segmentation else len(sentences)
    
    print(f"\n✓ Procesamiento completado:")
    print(f"  - Oraciones procesadas: {total_sentences}")
    print(f"  - Entidades encontradas: {len(all_entities_list)}")
    print(f"  - Relaciones extraídas: {len(all_relations)}")
    
    return all_relations, all_entities_list

# Procesar todo el documento
# Recomendado: use_spacy_segmentation=True (usa el segmentador nativo de spaCy)
# Alternativo: use_spacy_segmentation=False (usa NLTK sent_tokenize)
all_relations, all_entities = process_full_text(
    cleaned_text, 
    nlp, 
    max_chars=None,  # None = procesar todo el texto
    use_spacy_segmentation=True  # Usar segmentador de spaCy (recomendado)
)


✓ Procesando todo el texto: 409,982 caracteres

📝 Usando segmentador de spaCy (método recomendado)
Procesando texto completo con spaCy...
✓ Texto dividido en 2052 oraciones por spaCy

Procesando 2052 oraciones...
  Procesando oración 0/2052 (0.0%)
  Procesando oración 100/2052 (4.9%)
  Procesando oración 200/2052 (9.7%)
  Procesando oración 300/2052 (14.6%)
  Procesando oración 400/2052 (19.5%)
  Procesando oración 500/2052 (24.4%)
  Procesando oración 600/2052 (29.2%)
  Procesando oración 700/2052 (34.1%)
  Procesando oración 800/2052 (39.0%)
  Procesando oración 900/2052 (43.9%)
  Procesando oración 1000/2052 (48.7%)
  Procesando oración 1100/2052 (53.6%)
  Procesando oración 1200/2052 (58.5%)
  Procesando oración 1300/2052 (63.4%)
  Procesando oración 1400/2052 (68.2%)
  Procesando oración 1500/2052 (73.1%)
  Procesando oración 1600/2052 (78.0%)
  Procesando oración 1700/2052 (82.8%)
  Procesando oración 1800/2052 (87.7%)
  Procesando oración 1900/2052 (92.6%)
  Procesando oración 2

## 9. Analizar resultados


In [32]:
# Crear DataFrames para análisis
relations_df = pd.DataFrame(all_relations)
entities_df_full = pd.DataFrame(all_entities)

print("=" * 60)
print("RESUMEN DE EXTRACCIÓN")
print("=" * 60)
print(f"\nTotal de entidades únicas: {entities_df_full['text'].nunique()}")
print(f"\nTipos de entidades:")
print(entities_df_full['label'].value_counts())
print(f"\nTotal de relaciones extraídas: {len(relations_df)}")
print(f"\nTipos de relaciones más comunes:")
if len(relations_df) > 0:
    print(relations_df['relation'].value_counts().head(10))


RESUMEN DE EXTRACCIÓN

Total de entidades únicas: 2050

Tipos de entidades:
label
MISC    3339
ORG      461
LOC      189
PER      109
Name: count, dtype: int64

Total de relaciones extraídas: 9789

Tipos de relaciones más comunes:
relation
de de        1110
de            675
en de         307
ofrecen       297
•             289
para para     266
para en       216
del           211
de con        204
con con       201
Name: count, dtype: int64


## 10. Visualizar relaciones extraídas


In [33]:
# Mostrar algunas relaciones de ejemplo
print("\nEjemplos de relaciones extraídas:")
print("=" * 60)
if len(relations_df) > 0:
    for idx, row in relations_df.head(15).iterrows():
        print(f"\n{idx+1}. {row['subject']} -[{row['relation']}]-> {row['object']}")
        print(f"   Tipo sujeto: {row['subject_type']}, Tipo objeto: {row['object_type']}")
        print(f"   Oración: {row['sentence'][:100]}...")
else:
    print("No se encontraron relaciones. Esto puede deberse a:")
    print("- El texto no tiene estructuras sintácticas claras")
    print("- Necesitamos ajustar los patrones de extracción")
    print("- El modelo de spaCy puede necesitar más contexto")



Ejemplos de relaciones extraídas:

1. AWS Documento -[técnico]-> Descripción general de Amazon Web Services Copyright
   Tipo sujeto: MISC, Tipo objeto: MISC
   Oración: AWS Documento técnico Descripción general de Amazon Web Services Copyright © 2024 Amazon Web Service...

2. AWS Documento -[de]-> Amazon Web Services
   Tipo sujeto: MISC, Tipo objeto: MISC
   Oración: AWS Documento técnico Descripción general de Amazon Web Services Copyright © 2024 Amazon Web Service...

3. Descripción general de Amazon Web Services Copyright -[©]-> Amazon Web Services
   Tipo sujeto: MISC, Tipo objeto: MISC
   Oración: AWS Documento técnico Descripción general de Amazon Web Services Copyright © 2024 Amazon Web Service...

4. Amazon Web Services AWS Documento -[técnico]-> Descripción general de Amazon Web Services
   Tipo sujeto: MISC, Tipo objeto: MISC
   Oración: Descripción general de Amazon Web Services AWS Documento técnico Descripción general de Amazon Web S...

5. Amazon Web Services AWS Docum

## 11. Normalización de datos

**IMPORTANTE**: Esta es una etapa crucial antes de guardar. La normalización:
- Elimina duplicados de entidades (ej: "Link" y "Link's" → misma entidad)
- Agrupa relaciones similares usando stemming
- Mejora la calidad de los datos para cargar en Neo4j

### 11.1. Stemming de relaciones
Reduce palabras a su raíz para agrupar variantes (ej: "characters" → "charact")

### 11.2. Matching de entidades
Encuentra entidades que se refieren a lo mismo usando distancia de Levenshtein y slugify



In [34]:
import numpy as np

def levenshtein_distance(seq1, seq2):
    """Calcula la distancia de Levenshtein entre dos strings"""
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x - 1] == seq2[y - 1]:
                matrix[x, y] = min(matrix[x - 1, y] + 1, matrix[x - 1, y - 1], matrix[x, y - 1] + 1)
            else:
                matrix[x, y] = min(matrix[x - 1, y] + 1, matrix[x - 1, y - 1] + 1, matrix[x, y - 1] + 1)
    return int(matrix[size_x - 1, size_y - 1])

def normalize_relations(relations_df):
    """Normaliza las relaciones usando stemming"""
    if len(relations_df) == 0:
        return relations_df
    
    # Aplicar stemming a las relaciones
    relations_df_normalized = relations_df.copy()
    relations_df_normalized['relation_stemmed'] = relations_df_normalized['relation'].apply(
        lambda x: stemmer.stem(str(x).lower()) if pd.notna(x) else ''
    )
    
    return relations_df_normalized

def normalize_entities(entities_df):
    """Normaliza nombres de entidades usando slugify y agrupa similares"""
    if len(entities_df) == 0:
        return entities_df, {}
    
    entities_df_normalized = entities_df.copy()
    
    # Crear versiones normalizadas (slugify)
    entities_df_normalized['normalized_name'] = entities_df_normalized['text'].apply(
        lambda x: slugify(str(x).lower()) if pd.notna(x) else ''
    )
    
    # Crear diccionario de matching: {nombre_normalizado: [lista_de_variantes]}
    entity_mapping = {}
    for idx, row in entities_df_normalized.iterrows():
        normalized = row['normalized_name']
        original = row['text']
        
        if normalized not in entity_mapping:
            entity_mapping[normalized] = []
        entity_mapping[normalized].append(original)
    
    # Encontrar entidades similares usando distancia de Levenshtein
    similar_entities = {}
    entities_list = entities_df_normalized['normalized_name'].unique()
    
    for i, ent1 in enumerate(entities_list):
        if ent1 in similar_entities:
            continue
        similar_entities[ent1] = [ent1]
        
        for ent2 in entities_list[i+1:]:
            if len(ent1) > 3 and len(ent2) > 3:  # Solo comparar strings significativos
                distance = levenshtein_distance(ent1, ent2)
                if distance <= 2:  # Distancia pequeña = entidades similares
                    similar_entities[ent1].append(ent2)
    
    return entities_df_normalized, similar_entities

# Aplicar normalización
print("=" * 60)
print("NORMALIZANDO DATOS")
print("=" * 60)

print("\nNormalizando relaciones...")
if len(relations_df) > 0:
    relations_df_normalized = normalize_relations(relations_df)
    print(f"✓ {len(relations_df_normalized)} relaciones normalizadas")
    print("\nEjemplos de relaciones normalizadas:")
    print(relations_df_normalized[['relation', 'relation_stemmed']].head(10))
else:
    relations_df_normalized = relations_df
    print("⚠ No hay relaciones para normalizar")

print("\n" + "=" * 60)
print("Normalizando entidades...")
if len(entities_df_full) > 0:
    entities_df_normalized, entity_mapping = normalize_entities(entities_df_full)
    print(f"✓ {len(entities_df_normalized)} entidades normalizadas")
    print(f"✓ {len(entity_mapping)} grupos de entidades únicas")
    print("\nEjemplos de entidades agrupadas (primeros 5 grupos):")
    for i, (normalized, variants) in enumerate(list(entity_mapping.items())[:5]):
        print(f"  {normalized}: {variants}")
else:
    entities_df_normalized = entities_df_full
    entity_mapping = {}
    print("⚠ No hay entidades para normalizar")

print("\n" + "=" * 60)
print("✓ Normalización completada")
print("=" * 60)
print("Los datos normalizados se usarán en el siguiente paso (Guardar resultados)")


NORMALIZANDO DATOS

Normalizando relaciones...
✓ 9789 relaciones normalizadas

Ejemplos de relaciones normalizadas:
  relation relation_stemmed
0  técnico          técnico
1       de               de
2        ©                ©
3  técnico          técnico
4       de               de
5       de               de
6       de               de
7        :                :
8        :                :
9        :                :

Normalizando entidades...
✓ 4098 entidades normalizadas
✓ 2003 grupos de entidades únicas

Ejemplos de entidades agrupadas (primeros 5 grupos):
  aws-documento: ['aws-documento', 'aws-documentos']
  descripcion-general-de-amazon-web-services-copyright: ['descripcion-general-de-amazon-web-services-copyright']
  amazon-web-services: ['amazon-web-services']
  all-rights-reserved: ['all-rights-reserved']
  descripcion-general-de-amazon-web-services-aws-documento: ['descripcion-general-de-amazon-web-services-aws-documento']

✓ Normalización completada
Los datos normalizados

## 12. Guardar resultados

Guardamos los resultados en formato CSV y JSON para posterior procesamiento.

### Configuración

**Variable `USE_NORMALIZED_DATA`**: 
- `True` (recomendado): Guarda datos normalizados (elimina duplicados, mejor calidad)
- `False`: Guarda datos originales sin normalizar

Los archivos siempre se guardan con los mismos nombres (`entities.csv` y `relations.csv`) para que el script `pdf_data_loading.py` no tenga que buscar entre diferentes versiones.


In [36]:
# ============================================================================
# CONFIGURACIÓN: Bandera para usar datos normalizados o no
# ============================================================================
USE_NORMALIZED_DATA = True  # Cambia a False para usar datos sin normalizar

# Crear directorio de salida si no existe
output_dir = "pdf_extraction_output"
os.makedirs(output_dir, exist_ok=True)

print("=" * 60)
print("GUARDANDO RESULTADOS")
print("=" * 60)
print(f"📌 Usando datos: {'NORMALIZADOS' if USE_NORMALIZED_DATA else 'ORIGINALES (sin normalizar)'}")
print("=" * 60)

# Determinar qué datos usar según la bandera
if USE_NORMALIZED_DATA:
    # Verificar que existen datos normalizados
    if 'entities_df_normalized' in globals() and len(entities_df_normalized) > 0:
        entities_to_save = entities_df_normalized.drop_duplicates(subset=['normalized_name', 'label'])
        entities_source = "normalizados"
        print("✓ Usando entidades normalizadas")
    else:
        entities_to_save = entities_df_full.drop_duplicates(subset=['text', 'label']) if len(entities_df_full) > 0 else pd.DataFrame()
        entities_source = "originales (no hay normalizados)"
        print("⚠ No hay datos normalizados disponibles. Usando datos originales.")
else:
    entities_to_save = entities_df_full.drop_duplicates(subset=['text', 'label']) if len(entities_df_full) > 0 else pd.DataFrame()
    entities_source = "originales"
    print("✓ Usando entidades originales (sin normalizar)")

if USE_NORMALIZED_DATA:
    if 'relations_df_normalized' in globals() and len(relations_df_normalized) > 0:
        relations_to_save = relations_df_normalized
        relations_source = "normalizadas"
        print("✓ Usando relaciones normalizadas")
    else:
        relations_to_save = relations_df if len(relations_df) > 0 else pd.DataFrame()
        relations_source = "originales (no hay normalizadas)"
        print("⚠ No hay relaciones normalizadas disponibles. Usando datos originales.")
else:
    relations_to_save = relations_df if len(relations_df) > 0 else pd.DataFrame()
    relations_source = "originales"
    print("✓ Usando relaciones originales (sin normalizar)")

# Guardar entidades (siempre con el mismo nombre: entities.csv)
if len(entities_to_save) > 0:
    # Si son normalizadas, usar el nombre normalizado como texto principal
    if USE_NORMALIZED_DATA and 'normalized_name' in entities_to_save.columns:
        # Crear una copia con el nombre normalizado como texto principal
        entities_output = entities_to_save.copy()
        entities_output['text'] = entities_output['normalized_name']
        entities_output = entities_output.drop(columns=['normalized_name'], errors='ignore')
    else:
        entities_output = entities_to_save
    
    entities_output.to_csv(f"{output_dir}/entities.csv", index=False)
    print(f"\n✓ Entidades guardadas: {output_dir}/entities.csv ({len(entities_output)} entidades, {entities_source})")
else:
    print("\n⚠ No hay entidades para guardar")

# Guardar relaciones (siempre con el mismo nombre: relations.csv)
if len(relations_to_save) > 0:
    # Si son normalizadas, mantener la estructura pero eliminar columnas extra
    if USE_NORMALIZED_DATA and 'relation_stemmed' in relations_to_save.columns:
        relations_output = relations_to_save.drop(columns=['relation_stemmed'], errors='ignore')
    else:
        relations_output = relations_to_save
    
    relations_output.to_csv(f"{output_dir}/relations.csv", index=False)
    print(f"✓ Relaciones guardadas: {output_dir}/relations.csv ({len(relations_output)} relaciones, {relations_source})")
else:
    print("⚠ No hay relaciones para guardar")

# Guardar mapeo de entidades si existe (solo si se normalizó)
if USE_NORMALIZED_DATA and 'entity_mapping' in globals() and len(entity_mapping) > 0:
    with open(f"{output_dir}/entity_mapping.json", 'w', encoding='utf-8') as f:
        json.dump(entity_mapping, f, ensure_ascii=False, indent=2)
    print(f"✓ Mapeo de entidades guardado: {output_dir}/entity_mapping.json")

# Guardar en formato JSON también
results = {
    'entities': entities_to_save.to_dict('records') if len(entities_to_save) > 0 else [],
    'relations': relations_to_save.to_dict('records') if len(relations_to_save) > 0 else [],
    'metadata': {
        'total_chars': len(cleaned_text),
        'total_entities': len(entities_to_save),
        'total_relations': len(relations_to_save),
        'unique_entities': entities_to_save['text'].nunique() if len(entities_to_save) > 0 and 'text' in entities_to_save.columns else 0,
        'data_source': entities_source,
        'normalized': USE_NORMALIZED_DATA
    }
}

# Agregar datos originales como referencia si se usaron normalizados
if USE_NORMALIZED_DATA:
    if len(entities_df_full) > 0:
        results['entities_original'] = entities_df_full.drop_duplicates(subset=['text', 'label']).to_dict('records')
    if len(relations_df) > 0:
        results['relations_original'] = relations_df.to_dict('records')

with open(f"{output_dir}/extraction_results.json", 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✓ Resultados completos guardados: {output_dir}/extraction_results.json")



GUARDANDO RESULTADOS
📌 Usando datos: NORMALIZADOS
✓ Usando entidades normalizadas
✓ Usando relaciones normalizadas

✓ Entidades guardadas: pdf_extraction_output/entities.csv (2087 entidades, normalizados)
✓ Relaciones guardadas: pdf_extraction_output/relations.csv (9789 relaciones, normalizadas)
✓ Mapeo de entidades guardado: pdf_extraction_output/entity_mapping.json

✓ Resultados completos guardados: pdf_extraction_output/extraction_results.json
